In [1]:
# ติดตั้ง AutoGluon และ library สำหรับดาวน์โหลดข้อมูลจาก Kaggle
# ขั้นตอนนี้อาจใช้เวลา 2-3 นาที
%pip install --upgrade "autogluon[all]" opendatasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [14]:
import opendatasets as od
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# ระบุ URL ของชุดข้อมูลบน Kaggle
dataset_url = 'https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption'

# ดาวน์โหลดข้อมูล (ระบบจะถามหา Kaggle Username และ Key)
od.download(dataset_url)

# กำหนด Path ของไฟล์ข้อมูล
data_dir = '/content/hourly-energy-consumption'

# โหลดข้อมูลด้วย pandas
# และสั่งให้ pandas แปลงคอลัมน์ Datetime เป็น datetime object ทันที
df = pd.read_csv(f'{data_dir}/PJME_hourly.csv', parse_dates=['Datetime'])

# แสดงขนาดและตัวอย่างข้อมูล
print(f"ขนาดข้อมูล: {df.shape}")
print("\nตัวอย่างข้อมูล 5 แถวแรก:")
df.head()


Skipping, found downloaded files in "./hourly-energy-consumption" (use force=True to force download)
ขนาดข้อมูล: (145366, 2)

ตัวอย่างข้อมูล 5 แถวแรก:


,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


In [47]:
# --- ขั้นตอนที่ 3 (เวอร์ชันอัปเดต) ---

def create_calendar_features(data, timestamp_col='Datetime'):
    """
    สร้าง features จากคอลัมน์เวลาที่ระบุ
    - data: DataFrame ที่ต้องการสร้างฟีเจอร์
    - timestamp_col: ชื่อของคอลัมน์ที่เป็น timestamp
    """
    df = data.copy()
    # ใช้ชื่อคอลัมน์จากพารามิเตอร์ timestamp_col แทนการ hardcode 'Datetime'
    df['hour'] = df[timestamp_col].dt.hour
    df['dayofweek'] = df[timestamp_col].dt.dayofweek
    df['quarter'] = df[timestamp_col].dt.quarter
    df['month'] = df[timestamp_col].dt.month
    df['year'] = df[timestamp_col].dt.year
    df['dayofyear'] = df[timestamp_col].dt.dayofyear
    df['dayofmonth'] = df[timestamp_col].dt.day
    df['weekofyear'] = df[timestamp_col].dt.isocalendar().week.astype(int)
    return df

# ตอนสร้างฟีเจอร์สำหรับข้อมูล train ครั้งแรก ก็เรียกใช้แบบนี้ (ใช้ค่า default 'Datetime')
df = create_calendar_features(df, timestamp_col='Datetime')

# โค้ดส่วนที่เหลือในขั้นตอนที่ 4 และ 5 เหมือนเดิม...
# ...


In [34]:
# เพิ่มคอลัมน์ item_id สำหรับระบุซีรีส์
df['item_id'] = 'PJME'

# เปลี่ยนชื่อคอลัมน์ target เพื่อความชัดเจน
df.rename(columns={'PJME_MW': 'target'}, inplace=True)

# สร้าง TimeSeriesDataFrame จาก DataFrame ที่รวมทุกอย่างไว้แล้ว
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="Datetime"
)

print("ตัวอย่าง TimeSeriesDataFrame สำหรับ Train:")
train_data.head()


ตัวอย่าง TimeSeriesDataFrame สำหรับ Train:


target  hour  dayofweek  quarter  month  year  \
item_id timestamp                                                             
PJME    2002-12-31 01:00:00  26498.0     1          1        4     12  2002   
        2002-12-31 02:00:00  25147.0     2          1        4     12  2002   
        2002-12-31 03:00:00  24574.0     3          1        4     12  2002   
        2002-12-31 04:00:00  24393.0     4          1        4     12  2002   
        2002-12-31 05:00:00  24860.0     5          1        4     12  2002   

                             dayofyear  dayofmonth  weekofyear  
item_id timestamp                                               
PJME    2002-12-31 01:00:00        365          31           1  
        2002-12-31 02:00:00        365          31           1  
        2002-12-31 03:00:00        365          31           1  
        2002-12-31 04:00:00        365          31           1  
        2002-12-31 05:00:00        365          31           1

In [36]:
prediction_length = 48
eval_metric = "MASE"
time_limit = 600

# สร้าง Predictor และระบุชื่อของ Covariates ที่จะใช้
predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path="ag_models_energy_corrected",
    target="target",
    eval_metric=eval_metric,
    known_covariates_names=[
        "hour", "dayofweek", "quarter", "month",
        "year", "dayofyear", "dayofmonth", "weekofyear"
    ],
    freq='H'
)

# เริ่มฝึกโมเดล โดยส่งแค่ train_data ที่มีข้อมูลครบถ้วนเข้าไป
predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=time_limit,
)


/usr/local/lib/python3.11/dist-packages/autogluon/timeseries/predictor.py:198: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = pd.tseries.frequencies.to_offset(self.freq)
Frequency 'H' stored as 'h'
Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to '/content/ag_models_energy_corrected'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       10.72 GB / 12.67 GB (84.6%)
Disk Space Avail:   64.98 GB / 107.72 GB (60.3%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'freq': 'h',
 'hyperparameters': 'light',
 'known_covariates_names': ['hour',
                            'dayofweek',
                      

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/191M [00:00<?, ?B/s]

	-1.4504       = Validation score (-MASE)
	23.78   s     = Training runtime
	4.56    s     = Validation (prediction) runtime
Training timeseries model TemporalFusionTransformer. Training for up to 244.2s of the 488.4s of remaining time.
	-1.5280       = Validation score (-MASE)
	223.22  s     = Training runtime
	0.22    s     = Validation (prediction) runtime
Fitting simple weighted ensemble.
	Ensemble weights: {'RecursiveTabular': 1.0}
	-0.8740       = Validation score (-MASE)
	0.75    s     = Training runtime
	2.69    s     = Validation (prediction) runtime
Training complete. Models trained: ['Naive', 'SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'ETS', 'Theta', 'Chronos[bolt_small]', 'TemporalFusionTransformer', 'WeightedEnsemble']
Total runtime: 334.63 s
Best model: RecursiveTabular
Best model score: -0.8740


In [48]:
# --- ขั้นตอนที่ 6: พยากรณ์อนาคตและสร้างไฟล์ Submission (วิธีที่ถูกต้องที่สุด) ---

# --- 1. สร้าง DataFrame ของอนาคตด้วยฟังก์ชันของ AutoGluon ---
# ผลลัพธ์ที่ได้จะมีคอลัมน์ชื่อ 'item_id' และ 'timestamp'
future_data_frame = predictor.make_future_data_frame(train_data)
df_for_features = future_data_frame.reset_index()


# --- 2. สร้างฟีเจอร์โดยใช้ฟังก์ชันที่ยืดหยุ่นของเรา ---
# !!! จุดแก้ไข: เราไม่ต้อง rename คอลัมน์แล้ว !!!
# แค่บอกฟังก์ชันของเราว่าให้ใช้คอลัมน์ที่ชื่อ 'timestamp'
future_known_covariates_df = create_calendar_features(
    df_for_features,
    timestamp_col='timestamp'  # บอกฟังก์ชันให้ใช้คอลัมน์นี้
)


# --- 3. ทำการพยากรณ์ ---
# ตอนนี้ future_known_covariates_df มีทั้งคอลัมน์ 'timestamp' และฟีเจอร์อื่นๆ ครบถ้วน
# จึงสามารถส่งเข้า predict ได้โดยตรง
predictions = predictor.predict(
    train_data,
    known_covariates=future_known_covariates_df
)


# --- 4. สร้างไฟล์ Submission (เหมือนเดิม) ---
submission = predictions.reset_index()[['timestamp', 'mean']]
submission.rename(columns={'timestamp': 'Datetime', 'mean': 'predicted_demand'}, inplace=True)
submission['predicted_demand'] = submission['predicted_demand'].apply(lambda x: max(0, x))

print("ตัวอย่างไฟล์ Submission:")
submission.head()

# บันทึกเป็นไฟล์ CSV
submission.to_csv('submission_energy.csv', index=False)

print("\nสร้างไฟล์ submission_energy.csv เรียบร้อยแล้ว!")


data with frequency 'IRREG' has been resampled to frequency 'h'.
data with frequency 'IRREG' has been resampled to frequency 'h'.
Model not specified in predict, will default to the model with the best validation score: RecursiveTabular


ตัวอย่างไฟล์ Submission:

สร้างไฟล์ submission_energy.csv เรียบร้อยแล้ว!


In [49]:
submission

,Datetime,predicted_demand
0,2018-08-03 01:00:00,32935.820312
1,2018-08-03 02:00:00,31008.458984
2,2018-08-03 03:00:00,29662.728516
3,2018-08-03 04:00:00,29022.214844
4,2018-08-03 05:00:00,29130.972656
5,2018-08-03 06:00:00,30490.500000
6,2018-08-03 07:00:00,32379.955078
7,2018-08-03 08:00:00,34757.718750
8,2018-08-03 09:00:00,36818.414062
9,2018-08-03 10:00:00,38737.910156
